In [23]:
import pandas
import numpy as np
import math
import itertools
import random
from datetime import datetime

In [24]:
airports_fname = 'data/PrediqtAirports.csv'
training_fname = 'data/PrediqtTrainData.csv'

out_training = 'my_data/training.{:03d}.csv'
out_validation = 'my_data/validation.{:03d}.csv'
out_test = 'my_data/test.{:03d}.csv'

In [25]:
with open(airports_fname, 'r') as f:
    airports = pandas.read_csv(airports_fname)
for key in ('LATITUDE', 'LONGITUDE'):
    print key
    airports[key] = airports[key].apply(math.radians)

LATITUDE
LONGITUDE


In [26]:
ONE_DAY = 60 * 60 * 24

BINS = {}
BINS['PRICE'] = (
    0.0, 42.18, 54.600000000000001, 65.359999999999999, 
    75.280000000000001, 84.540000000000006, 93.620000000000005, 
    101.38, 109.23, 117.3, 124.98, 132.22, 139.58000000000001, 
    145.97999999999999, 151.97999999999999, 158.28999999999999, 
    163.84999999999999, 169.80000000000001, 174.24000000000001, 
    179.97999999999999, 185.36000000000001, 190.19, 195.62, 
    199.97999999999999, 204.97999999999999, 209.97999999999999, 
    215.34, 219.97999999999999, 225.08000000000001, 229.97999999999999, 
    235.36000000000001, 239.97999999999999, 245.40000000000001, 250.0, 
    256.05000000000001, 260.98000000000002, 268.86000000000001, 273.87, 
    279.98000000000002, 287.75999999999999, 294.49000000000001, 
    300.98000000000002, 309.98000000000002, 319.26999999999998, 
    326.98000000000002, 335.69, 343.48000000000002, 353.5, 
    362.98000000000002, 374.48000000000002, 385.13999999999999, 
    397.82999999999998, 409.98000000000002, 423.30000000000001, 
    437.98000000000002, 451.60000000000002, 466.51999999999998, 480.87, 
    496.33999999999997, 510.24000000000001, 526.02999999999997, 
    539.98000000000002, 550.28999999999996, 568.98000000000002, 
    579.98000000000002, 599.46000000000004, 612.47000000000003, 
    629.98000000000002, 648.98000000000002, 664.0, 679.98000000000002, 
    699.09000000000003, 709.98000000000002, 727.07000000000005, 
    739.98000000000002, 756.72000000000003, 769.98000000000002, 
    786.98000000000002, 799.98000000000002, 810.92999999999995, 
    825.08000000000004, 833.80999999999995, 848.0, 856.07000000000005, 
    867.98000000000002, 874.73000000000002, 886.16999999999996, 
    896.10000000000002, 902.10000000000002, 909.98000000000002, 
    919.98000000000002, 929.98000000000002, 939.98000000000002, 
    949.98000000000002, 959.98000000000002, 969.98000000000002, 
    979.98000000000002, 989.98000000000002, 999.98000000000002, 1009.98, 
    1019.98, 1029.98, 1039.98, 1049.98, 1061.98, 1073.29, 1081.23, 
    1091.98, 1103.5, 1115.98, 1127.98, 1138.49, 1149.98, 1161.98, 
    1175.98, 1188.98, 1199.98, 1216.4300000000001, 1229.98, 1245.98, 
    1259.98, 1277.0899999999999, 1289.98, 1307.98, 1319.98, 1339.98, 
    1359.98, 1379.98, 1399.98, 1419.98, 1440.98, 1462.98, 1489.98, 
    1519.98, 1554.98, 1589.98, 1639.98, 1699.98, 1778.98, 1886.98, 
    2052.98, 10099.838400000001)
BINS['STI'] = (  
    0.00000000e+00,   3.00000000e+00,   4.00000000e+00,
    7.00000000e+00,   9.00000000e+00,   1.20000000e+01,
    1.50000000e+01,   1.90000000e+01,   2.40000000e+01,
    3.30000000e+01,   1.71990000e+04)
BINS['STO'] = ( 
    -1.68070000e+04,   1.50000000e+01,   2.90000000e+01,
    4.30000000e+01,   6.10000000e+01,   8.10000000e+01,
    1.12000000e+02,   1.46000000e+02,   1.80000000e+02,
    2.15000000e+02,   4.37000000e+02)
BINS['OTI'] = (-2.,  27.,  43.,  60.,  79.,  99., 131., 166., 202., 235., 451.)

MOST_COMMON = {}
MOST_COMMON['MARKETS'] = (
    '**', 'UK', 'US', 'FR', 'IT', 'DE', 'ES', 'SK', 'TR', 'AT', 'PT', 'PL', 
    'RU', 'IE', 'HU', 'CZ', 'GR', 'BR', 'BG', 'NL', 'RO', 'CH', 'TW', 'HK', 
    'MT', 'IL', 'BE', 'SA', 'CA', 'KW', 'MO', 'SI', 'QA', 'KZ', 'BH', 'OM', 
    'IS', 'BY', 'GE', 'ME', 'MD', 'LB', 'AZ', 'AD', 'SM', 'VA', 'HR', 'LI', 
    'BA', 'GL', 'FO', 'MK', 'KO', 'BV', 'MN', 'LU', 'WF', 'TF', 'BN', 'WS', 
    'GU', 'PW', 'PG', 'PN', 'SB', 'MP', 'FJ', 'FM', 'VU', 'NU', 'CK', 'CC', 
    'CX', 'KH', 'TV', 'TO', 'TL', 'TK', 'AS', 'LK', 'MV', 'BT', 'BQ', 'GS', 
    'HM', 'MH', 'UM', 'MS', 'NF', 'CW', 'SX', 'KI', 'SJ', 'LA', 'AN', 'AQ', 
    'IO', 'MC', 'RS')
MOST_COMMON['ORIGIN'] = (
    'MAN', 'LHR', 'FRA', 'LGW', 'CDG', 'AMS', 'MAD', 'BHX', 'DUS', 'MUC', 
    'STN', 'MXP', 'BRU', 'GLA', 'SVO', 'FCO', 'HAM', 'BCN', 'GRU', 'IST', 
    'GIG', 'ZRH', 'MEX', 'STR', 'CGN', 'EMA', 'YYZ', 'DUB', 'CUN', 'TXL', 
    'DME', 'CPH', 'NCL', 'ATH', 'VCE', 'VIE', 'LIS', 'LAX', 'WAW', 'BUD', 
    'GVA', 'BLQ', 'LED', 'HAJ', 'LIN', 'BRS', 'EDI', 'JFK', 'HEL', 'PRG', 
    'ORY', 'ARN', 'LBA', 'SFO', 'EZE', 'LPL', 'OSL', 'SXF', 'BIO', 'MSP', 
    'OPO', 'LYS', 'TFS', 'IAH', 'DFW', 'BOG', 'BWI', 'BLL', 'MRS', 'RIX', 
    'BSL', 'PDX', 'TLL', 'BSB', 'LTN', 'SJO', 'SEA', 'BKK', 'TLV', 'VCP', 
    'OAK', 'POA', 'PMI', 'VNO', 'SOF', 'SIN', 'SCL', 'BGY', 'AYT', 'LEJ', 
    'REC', 'AAL', 'TLS', 'YUL', 'NCE', 'LJU', 'CNF', 'MCO', 'FOR', 'YVR', 
    'LAS', 'KBP', 'FLR', 'BEG', 'LCA', 'BRE', 'OTP', 'ZAG', 'BFS', 'CWL', 
    'CPT', 'SJC', 'KRK', 'DEN', 'BUR', 'TRN', 'BOS', 'JED', 'ORD', 'VRN', 
    'JNB', 'SKG', 'AGP', 'GOT', 'SSA', 'LIM', 'GDL', 'CCS', 'HAV', 'NAP', 
    'ONT', 'CAI', 'SYD', 'PHX', 'LCY', 'NUE', 'AUS', 'CWB', 'ISP', 'FLL', 
    'MEL', 'EIN', 'EWR', 'SNA', 'HER', 'ALC', 'DLM', 'SAN', 'SDQ')
MOST_COMMON['DESTINATION_COUNTRY'] = (
    'ES', 'US', 'CU', 'TH', 'UK', 'BR', 'TR', 'DE', 'MX', 'IT', 'GR', 'CA', 
    'DO', 'ZA', 'FR', 'PT', 'CR', 'NL', 'MU', 'MV', 'AR', 'RU', 'CY', 'IN', 
    'KE', 'PE', 'CH', 'CO', 'AU', 'EG', 'IE', 'SG', 'PL', float('NaN'), 'AE',
    'TZ', 'JP', 'AT', 'CL', 'JM', 'EC', 'BE', 'BB', 'CZ', 'PA', 'PH', 'ID', 
    'HU', 'CN')
MOST_COMMON['ORIGIN_COUNTRY'] = (
    'UK', 'DE', 'US', 'IT', 'ES', 'FR', 'BR', 'RU', 'NL', 'MX', 'BE', 'CH', 
    'CA', 'TR', 'DK', 'PL', 'GR', 'PT', 'IE', 'AT', 'SE', 'HU', 'FI', 'AR')
MOST_COMMON['DESTINATION'] = (
    'HAV', 'PMI', 'TFS', 'BKK', 'CUN', 'GIG', 'MCO', 'LAS', 'JFK', 'MAD', 
    'HKT', 'DLM', 'FRA', 'LHR', 'CPT', 'SJO', 'PUJ', 'MRU', 'LAX', 'BCN', 
    'YYZ', 'ACE', 'MLE', 'AYT', 'CDG', 'LPA', 'AMS', 'VRA', 'EZE', 'FUE', 
    'LIS', 'SFO', 'FCO', 'MAN', 'LGW', 'IBZ', 'LIM', 'HER', 'MEX', 'GRU', 
    'SDQ', 'LCA', 'SIN', 'IST', 'JNB', 'STN', 'WDH', 'DUB', 'ATH', 'ALC', 
    'MUC', 'MXP', 'BOG', 'AGP', 'NBO', 'SCL', 'MBJ', 'BGI', 'HNL', 'ZNZ', 
    'YUL', 'FAO', 'PTY', 'YVR', 'PRG', 'MAH', 'SEA', 'BUD', 'CFU', 'DXB', 
    'MBA', 'TXL', 'BRU', 'MIA', 'OPO', 'GVA', 'PFO', 'HRG', 'ZRH', 'ZTH', 
    'UIO', 'VIE', 'CPH', 'MNL', 'POP', 'BWI', 'MSP', 'HOG', 'FOR', 'DUS', 
    'DPS', 'DFW', 'GUA', 'IAH', 'MGA', 'EDI', 'SXF', 'NRT', 'SEZ', 'TLV', 
    'DEN', 'SYD', 'RHO', 'SVO', 'MEL', 'HKG', 'VCE', 'CAI', 'SAN', 'GLA', 
    'PNH', 'ARN', 'CGN', 'PDX', 'FLL', 'DEL', 'DME', 'BHX', 'PRN', 'FLR', 
    'EWR', 'KRK', 'SSA', 'GOI', 'BUR', 'OSL', 'SJU', 'BJL', 'PBI', 'SKG', 
    'LED', 'HAM', 'OAK', 'KUL', 'SOF', 'LIN', 'RTM', 'CMB', 'PHX', 'RAK', 
    'REC', 'BLQ', 'ICN', 'NCE', 'EIN', 'BOS', 'BGY', 'SJC', 'SNA', 'MJV', 
    'MLA', 'XRY', 'ORY', 'HEL', 'TFN', 'CTG', 'KEF', 'GYE', 'SGN', 'ORD', 
    'NAP', 'CCS', 'ONT', 'WAW', 'YYC', 'BSL', 'VCP', 'LYS', 'SLC', 'CIA', 
    'OTP', 'AKL', 'STR', 'BEG', 'SAL', 'BOM', 'BSB', 'BIO', 'RGN', 'SAW', 
    'MVD', 'KBP', 'VLC', 'DBV', 'SVQ', 'NAT', 'GRX', 'DUR', 'RUN', 'TRN', 
    'MRS', 'AUS', 'SSH', 'PSA', 'CNF', 'JTR', 'YQB', 'ZAG', 'KBV', 'PEK', 
    'RIX', 'JED', 'FNC', 'PVG', 'POA', 'TPE', 'KGS', 'LGA', 'ADB', 'BVA', 
    'REU', 'TLS', 'AER', 'ISB', 'GDL', 'ATL', 'AGA', 'CTA', 'CNX', 'BJV', 
    'LTN', 'BOJ', 'PER', 'VNO')

MAX = {}
MAX['TRAFFIC'] = 17435204.0
MAX['ORIGIN_LATITUDE'] = 1.51305442978
MAX['ORIGIN_LONGITUDE'] = 3.14744823323
MAX['DESTINATION_LATITUDE'] = 1.51305442978
MAX['DESTINATION_LONGITUDE'] = 3.16145255339
MAX['DISTANCE'] = 20141.4607366
MAX['PRICE'] = 10500

MIN = {}
MIN['TRAFFIC'] = 0.0
MIN['ORIGIN_LATITUDE'] = -0.966767766679
MIN['ORIGIN_LONGITUDE'] = -3.10543525513
MIN['DESTINATION_LATITUDE'] = -0.966767766679
MIN['DESTINATION_LONGITUDE'] = -3.17083736087
MIN['DISTANCE'] = 0.0
MIN['PRICE'] = 0.0

In [27]:
def convert(num, buckets, bucket_ranges=None,):
    """go from 0..1 to larger"""
    buckets = tuple(buckets)
    if bucket_ranges is None:
        bucket_ranges = itertools.izip(buckets, buckets[1:])
    step = 1 / float(len(buckets))
    cumsum = 0
    for low, high in bucket_ranges:
        if cumsum <= num < cumsum + step:
            t = (num - cumsum) / step
            return (1 - t) * low + t * high
        cumsum += step
    t = (num - (cumsum - step)) / step
    return (1 - t) * low + t * high

def reverse_convert(num, bin_key, _range=range(12), cache={}):
    """go from larger to 0..1"""
    key = (num, bin_key)
    try:
        return cache[key]
    except KeyError:
        pass
    
    # Binary search, because I couldn't spend time to figure it out properly... oh well
    buckets = BINS[bin_key]
    bucket_ranges = zip(buckets, buckets[1:])
    high, low = 0., 1.
    for __ in _range:
        mid = (high + low) / 2.
        result = convert(mid, buckets=buckets, bucket_ranges=bucket_ranges)
        if abs(result - num) < 0.001:
            break
        if result > num:
            low = mid
        else:
            high = mid
            
    cache[key] = mid
    return mid

In [28]:
def make_onehot(values, field, onehot=None, cache={}):
    try:
        values = values.split(';')
    except AttributeError:
        values = str(values).split(';')
    key = (tuple(values), field)
    try:
        return cache[key]
    except KeyError:
        pass
    
    most_common = MOST_COMMON[field]
    if onehot is None:
        #zeros_size = len(most_common) + (1 if excluded > 0 else 0)
        zeros_size = len(most_common) + (1)
        onehot = np.zeros((zeros_size,), dtype='bool')
    most_common_index = most_common.index
    for value in values:
        try:
            onehot[most_common_index(value)] = True
        except ValueError:
            onehot[zeros_size - 1] = True
    cache[key] = onehot
    return onehot

In [29]:
sin, cos, sqrt, atan2, radians = math.sin, math.cos, math.sqrt, math.atan2, math.radians
def distance(lat1, lon1, lat2, lon2, cache={}):
    key = (lat1, lon1, lat2, lon2)
    try:
        return cache[key]
    except KeyError:
        pass
    
    # Haversine
    delta_phi = lat2 - lat1
    delta_lambda = lon2 - lon1
    
    sdp2 = sin(delta_phi / 2.)
    sdl2 = sin(delta_lambda / 2.)
    a = sdp2 ** 2 + cos(lat1) * cos(lat2) * sdl2 ** 2
    dist = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    dist *= 0.31631270856  # shh
    cache[key] = dist
    return dist

In [30]:
def onehot_encode_date(date, cache={}):
    try:
        return cache[date]
    except KeyError:
        pass
    array = np.zeros((7 + 12 + 31,), dtype='bool')
    array[date.weekday() - 1] = True
    array[date.month + 7 - 1] = True
    array[date.day + 7 + 12 - 1] = True
    cache[date] = array
    return array

In [31]:
def lerp(value, field):
    minimum = MIN[field]
    return (value - minimum) / (MAX[field] - minimum)

In [32]:
radians = math.radians
def convert_row(row):
    outbound_dt = datetime.strptime(row['OUTBOUND_DATE'], '%Y-%m-%d')
    inbound_dt = datetime.strptime(row['INBOUND_DATE'], '%Y-%m-%d')
    search_dt = datetime.strptime(row['SEARCH_DATEHOUR'], '%Y-%m-%dT%H:%M:%S')
    outbound_date = onehot_encode_date(outbound_dt)
    inbound_date = onehot_encode_date(inbound_dt)
    
    sto = (outbound_dt - search_dt).total_seconds() / ONE_DAY
    sti = (inbound_dt - search_dt).total_seconds() / ONE_DAY
    oti = (inbound_dt - outbound_dt).total_seconds() / ONE_DAY
    search_to_outbound = reverse_convert(sto, 'STO')
    search_to_inbound = reverse_convert(sti, 'STI')
    outbound_to_inbound = reverse_convert(oti, 'OTI')
    
    markets = make_onehot(row['MARKETS'], 'MARKETS')
    
    row_origin = row['ORIGIN']
    origin = make_onehot(row_origin, 'ORIGIN')
    origin_data = airports[airports.AIRPORT == row_origin]
    origin_country = make_onehot(next(iter(origin_data['COUNTRY'])), 'ORIGIN_COUNTRY')
    origin_traffic = lerp(float(origin_data['TRAFFIC']), 'TRAFFIC')
    
    origin_latitude_rads = radians(float(origin_data['LATITUDE']))
    origin_longitude_rads = radians(float(origin_data['LONGITUDE']))
    origin_latitude = lerp(origin_latitude_rads, 'ORIGIN_LATITUDE')
    origin_longitude = lerp(origin_longitude_rads, 'ORIGIN_LONGITUDE')
    
    row_destination = row['DESTINATION']
    destination = make_onehot(row_destination, 'DESTINATION')
    destination_data = airports[airports.AIRPORT == row_destination]
    destination_country = make_onehot(next(iter(destination_data['COUNTRY'])), 'DESTINATION_COUNTRY')
    destination_traffic = lerp(float(destination_data['TRAFFIC']), 'TRAFFIC')
    
    destination_latitude_rads = radians(float(destination_data['LATITUDE']))
    destination_longitude_rads = radians(float(destination_data['LONGITUDE']))
    destination_latitude = lerp(destination_latitude_rads, 'DESTINATION_LATITUDE')
    destination_longitude = lerp(destination_longitude_rads, 'DESTINATION_LONGITUDE')
    
    dist = distance(
        origin_latitude_rads, origin_longitude_rads, 
        destination_latitude_rads, destination_longitude_rads
    )
    
    price = lerp(float(row['PRICE']), 'PRICE')
    
    return np.concatenate((
        outbound_date, inbound_date,
        markets,
        origin, origin_country,
        destination, destination_country,
        [search_to_outbound, search_to_inbound, outbound_to_inbound,
         origin_traffic, destination_traffic,
         origin_latitude, origin_longitude, destination_latitude, destination_longitude,
         dist, price]
    ))
    # return concatenation of values, arrays (as comma-separated string)

In [33]:
comma_join = ','.join
training_files = itertools.cycle(open(out_training.format(x), 'w') for x in xrange(100))
test_files = itertools.cycle(open(out_validation.format(x), 'w') for x in xrange(10))
valid_files = itertools.cycle(open(out_test.format(x), 'w') for x in xrange(10))
rand_choice = random.choice
with open(training_fname, 'r') as f:
    COLUMNS = next(f).strip().split(',')
    train_l, valid_l, test_l = [], [], []
    for i, line in enumerate(f):
        try:
            converted = convert_row(dict(itertools.izip(COLUMNS, line.strip().split(','))))
        except Exception as e:
            raw_input(repr(e))
            continue
        r = random.random()
        if r <= 0.8:
            write = train_l.append
        elif r <= 0.9:
            write = valid_l.append
        else:
            write = test_l.append
        write(comma_join(map(str, converted)).replace('.0,', ','))
        if not (i + 1) % 2500:
            print (i + 1)
            next(training_files).write('\n'.join(train_l) + '\n')
            next(valid_files).write('\n'.join(valid_l) + '\n')
            next(test_files).write('\n'.join(test_l) + '\n')
            train_l, valid_l, test_l = [], [], []
    next(training_files).write('\n'.join(train_l) + '\n')
    next(valid_files).write('\n'.join(valid_l) + '\n')
    next(test_files).write('\n'.join(test_l) + '\n')
    print 'Finished'

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500
35000
37500
40000
42500
45000
47500
50000
52500
55000
57500
60000
62500
65000
67500
70000
72500
75000
77500
80000
82500
85000
87500
90000
92500
95000
97500
100000
102500
105000
107500
110000
112500
115000
117500
120000
122500
125000
127500
130000
132500
135000
137500
140000
142500
145000
147500
150000
152500
155000
157500
160000
162500
165000
167500
170000
172500
175000
177500
180000
182500
185000
187500
190000
192500
195000
197500
200000
202500
205000
207500
210000
212500
215000
217500
220000
222500
225000
227500
230000
232500
235000
237500
240000
242500
245000
247500
250000
252500
255000
257500
260000
262500
265000
267500
270000
272500
275000
277500
280000
282500
285000
287500
290000
292500
295000
297500
300000
302500
305000
307500
310000
312500
315000
317500
320000
322500
325000
327500
330000
332500
335000
337500
340000
342500
345000
347500
350000
352500
355000
357500
360000
362500
365000
367500
370000
372500

In [34]:
datetime.strptime('2016-01-09T17:00:00', '%Y-%m-%dT%H:%M:%S')
s = '2016-01-03T00:00:00,2016-04-23,2016-05-20'
(datetime.now() - datetime.strptime('2016-01-09T17:00:00', '%Y-%m-%dT%H:%M:%S')).total_seconds() / ONE_DAY

55.83945409722222

In [35]:
np.concatenate((np.array([1, 2]), [3])).tolist()

[1, 2, 3]

In [40]:
import itertools
for i, f in enumerate(training_files):
    print i
    f.flush()
    os.fsync(f)
    if i > 150:
        break
for i, f in enumerate(test_files):
    print i
    f.flush()
    os.fsync(f)
    if i > 150:
        break
for i, f in enumerate(valid_files):
    print i
    f.flush()
    os.fsync(f)
    if i > 150:
        break
import os


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
0
1
